In [8]:
import genvarloader as gvl
import pooch
from pathlib import Path
import pyranges as pr
import polars as pl
from tempfile import NamedTemporaryFile
from loguru import logger
import numpy as np

In [2]:
logger.enable('genvarloader')

In [3]:
reference = "/carter/shared/genomes/homo_sapiens/ensembl_grch38.p13_v109/Homo_sapiens.GRCh38.dna.toplevel.fa.gz"

In [30]:
ds = gvl.Dataset.open(
    "/cellar/users/dlaub/projects/GenVarLoader/tests/data/phased_dataset.gvl", reference
).with_len(20)
center = (pl.col("chromEnd") + pl.col("chromStart")) // 2
sites = ds.regions.with_columns(POS=center, ALT=pl.lit("T")).rename({'chrom': "#CHROM"}).drop(
    "chromStart", "chromEnd"
)

2025-03-20 14:20:19.361 | INFO     | genvarloader._dataset._impl:open:225 - Loading reference genome into memory. This typically has a modest memory footprint (a few GB) and greatly improves performance.
2025-03-20 14:20:19.440 | INFO     | genvarloader._dataset._impl:open:284 - Opened dataset:
GVL store at /cellar/users/dlaub/projects/GenVarLoader/tests/data/phased_dataset.gvl
Is subset: False
# of regions: 8
# of samples: 3
Output length: ragged
Jitter: 0 (max: 0)
Deterministic: True
Sequence type: reference [haplotypes] annotated
Active tracks: None
Tracks available: None



In [31]:
cv_ds = gvl.DatasetWithSites(ds, sites)
cv_ds.rows

chrom,chromStart,chromEnd,POS,ALT
cat,i64,i64,i64,str
"""20""",14360,14380,14370,"""T"""
"""19""",1010685,1010705,1010695,"""T"""
"""19""",101,121,111,"""T"""
"""20""",17320,17340,17330,"""T"""
"""19""",1110686,1110706,1110696,"""T"""
"""19""",1210686,1210706,1210696,"""T"""
"""20""",1110686,1110706,1110696,"""T"""
"""20""",1234555,1234575,1234565,"""T"""


In [32]:
r_idx, s_idx = np.unravel_index(np.arange(len(cv_ds)), cv_ds.shape)
haps, flags = cv_ds[r_idx, s_idx]

In [33]:
# (b p), (b p l) -sum-> (b p)
np.testing.assert_equal(flags == 0, (haps.var_idxs == -2).sum(-1) == 1)